In [1]:
!pip install karateclub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
     |████████████████████████████████| 1.8 MB 18.6 MB/s 
     |████████████████████████████████| 24.1 MB 3.1 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
  Created wheel for karateclub: filename=karateclub-1.3.0-py3-none-any.whl size=98587 sha256=4089ec89bfd77b9714162930b64219c5cca6472dce86e7ca1c5696a503867878
  Stored in directory: /root/.cache/pip/wheels/5c/e7/85/fb61ed4735fef344e737483ac55d4900bbc9323fd500c25183
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149854 sha256=de1e442c58c45bb95238add7ab50d0e9986908f0c6f31435dcaeb8f38c8597f2
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built karateclub python-Levenshtein
  Attempting uninstall: gensim
    Found existing installation: g

In [2]:
import os
import csv
import pdb
import math
import time 
import pickle
import pickle
import random
import warnings
import datetime 
import collections
import numpy as np 
import pandas as pd  
import xgboost as xgb
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pylab as plt 
from matplotlib import rcParams   
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import time

# Algo import
from karateclub import DeepWalk

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
data_path = 'gdrive/My Drive/Major Project'

In [5]:
combined_tr = pd.read_csv(data_path+'/train_91.csv', index_col = [0])
print(combined_tr.shape)
combined_tr.head()

(902474, 3)


,src,dest,link
73132,35080,10701,0
404654,54809,75095,0
310475,67628,60404,0
301271,57253,462,0
332230,15167,47459,1


In [6]:
combined_te = pd.read_csv(data_path+'/test_91.csv', index_col = [0])
print(combined_te.shape)
combined_te.head()

(106363, 3)


,src,dest,link
487942,32198,62377,0
103900,6056,11650,1
78661,2040,22467,1
71074,143,0,1
382438,1609,5035,1


In [7]:
df_train = pd.read_csv(data_path+'/pos_train_91.csv', index_col = [0])
print(df_train.shape)
df_train.head()

(452474, 3)


,src,dest,link
496205,54500,16485,1
141147,3318,30547,1
483066,33514,4007,1
478351,28118,2641,1
505144,69985,61687,1


In [8]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph)

In [9]:
deepwalk = DeepWalk(walk_number=10, walk_length=16, dimensions=64)
start = time.time()
deepwalk.fit(train_graph)
print("Time taken for algo fit = ", time.time() - start)

Time taken for algo fit =  173.916846036911


In [10]:
embeddings = deepwalk.get_embedding()

In [11]:
emb_dict = {}
for node, emb in zip(train_graph.nodes(), embeddings):
  emb_dict[node] = emb
print(len(emb_dict))

75879


In [12]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined_tr.iterrows()):
  comb_emb = emb_dict[row['src']] * emb_dict[row['dest']]
  train_emb.append(comb_emb)
train_emb = np.array(train_emb)
print(train_emb.shape)
train_y = combined_tr['link']
print(len(train_y))

0it [00:00, ?it/s]

(902474, 64)
902474


In [13]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_te.iterrows()):
  comb_emb = emb_dict[row['src']] * emb_dict[row['dest']]
  test_emb.append(comb_emb)

test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_te['link'] 
print(len(test_y))

0it [00:00, ?it/s]

(106363, 64)
106363


In [19]:
pickle.dump((train_emb, train_y), open(data_path + '/epinions_deepwalk_train91.pkl', 'wb'))
pickle.dump((test_emb, test_y), open(data_path + '/epinions_deepwalk_test91.pkl', 'wb'))

In [15]:
train_emb, train_y = pickle.load(open(data_path + '/epinions_deepwalk_train91.pkl', 'rb'))
test_emb, test_y = pickle.load(open(data_path + '/epinions_deepwalk_test91.pkl', 'rb'))

In [ ]:
# train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.9, random_state = 25)

In [16]:
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for XGB fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for XGB fit =  1452.9662554264069
Time taken for train data precition =  5.360637664794922
Time taken for test data precition =  0.7386279106140137
Train Score 0.6590085462236484 test Score 0.6593915405655181


In [17]:
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for RandomForest fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for RandomForest fit =  924.9338510036469
Time taken for train data precition =  10.085272550582886
Time taken for test data precition =  1.2683825492858887
Train Score 0.660427983977525 test Score 0.6432329480095796


In [18]:
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for LGBM fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for LGBM fit =  43.70191144943237
Time taken for train data precition =  5.734234571456909
Time taken for test data precition =  0.6729907989501953
Train Score 0.6453303443267016 test Score 0.6576961372751552
